# calculate Indian Ocean dipole index


see:  
    
+ [https://climatedataguide.ucar.edu/climate-data/sst-data-noaa-extended-reconstruction-ssts-version-5](https://climatedataguide.ucar.edu/climate-data/sst-data-noaa-extended-reconstruction-ssts-version-5) 
+ [ftp://ftp.ncdc.noaa.gov/pub/data/cmb/ersst/v4/netcdf/](ftp://ftp.ncdc.noaa.gov/pub/data/cmb/ersst/v5/netcdf/)
+ http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc (for access via OpenDAP)

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
import os
import sys
import pathlib

import numpy as np
import pandas as pd
from datetime import (datetime, timedelta)
from dateutil import parser

In [3]:
import xarray as xr

In [4]:
HOME = pathlib.Path.home()

### access the ERSST dataset remotely (ESRL Thredds server: No need for download)

In [5]:
url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"

In [6]:
dset = xr.open_dataset(url, drop_variables=["time_bnds"])

### latitudes in ascending order 

In [7]:
dset = dset.sortby('lat')

### check what is the last month in the data 

In [8]:
dset.time[-1].data

array('2021-10-01T00:00:00.000000000', dtype='datetime64[ns]')

### keep only the data post 1950

In [9]:
dset = dset.sel(time=slice('1950', None))

### loads in memory 

In [10]:
dset = dset.compute()

### function to remove climatological average

In [11]:
def demean(x): 
    return x - x.sel(time=slice('1981','2010')).mean('time')

### calculates the anomalies WRT to climatology

In [12]:
sst_anoms = dset['sst'].groupby('time.month').apply(demean)

### get the west and east nodes 

In [13]:
EAST = sst_anoms.sel(lon=slice(90, 110), lat=slice(-10,0)).mean('lat').mean('lon')
WEST = sst_anoms.sel(lon=slice(50, 70), lat=slice(-10,10)).mean('lat').mean('lon')

In [14]:
time = EAST['time'].values

In [15]:
from scipy.stats import zscore

In [16]:
EAST = zscore(EAST.values.flatten())

In [17]:
WEST = zscore(WEST.values.flatten())

In [18]:
DMI = pd.DataFrame({'east':EAST, 'west':WEST}, index=time)

In [19]:
DMI = DMI.assign(dmi = DMI.west - DMI.east)

In [20]:
DMI

,east,west,dmi
1950-01-01,-1.282550,-1.287464,-0.004913
1950-02-01,-0.383997,-1.575100,-1.191103
1950-03-01,-0.576087,-1.276734,-0.700647
1950-04-01,-0.881950,-1.218672,-0.336722
1950-05-01,-1.212072,-1.399487,-0.187415
...,...,...,...
2021-06-01,1.429279,0.485073,-0.944206
2021-07-01,1.588399,0.799168,-0.789231
2021-08-01,1.436635,0.933145,-0.503491
2021-09-01,1.046687,1.008190,-0.038497


In [22]:
DMI = DMI.loc[:,['dmi']]

In [23]:
DMI.columns = ['IOD_monthly']

In [24]:
DMI.index.name = 'date'

In [25]:
DMI.index = DMI.index + pd.offsets.MonthEnd(0)

### calculates the seasonal anomalies 

In [26]:
DMIs = DMI.rolling(window=3, min_periods=3).mean()

In [27]:
DMIs.columns = ['IOD_seasonal']

In [28]:
DMI = pd.concat([DMI, DMIs], axis=1)

In [29]:
DMI.to_csv('./IOD.csv')